In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = '/content/gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
import xlrd
import statsmodels.api as sm
## gdal
from osgeo import gdal, gdal_array
import numpy as np
from numpy.linalg import inv
from scipy.stats.distributions import chi2
from itertools import chain
import matplotlib.pyplot as plt
from osgeo import ogr
## AI
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score, accuracy_score, f1_score, precision_score, recall_score

In [ ]:
import joblib

In [ ]:
randomforest = joblib.load("/content/gdrive/My Drive/Whanganui/classification_model.joblib") #import the AI-model

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
raster = gdal.Open('...tif') ##open your tiff file

In [ ]:
geot = raster.GetGeoTransform()
proj = raster.GetProjectionRef()
width = raster.RasterYSize; lenth = raster.RasterXSize; number = raster.RasterCount
container = np.ones((width, lenth, number),gdal_array.GDALTypeCodeToNumericTypeCode(raster.GetRasterBand(1).DataType))
for i in range(number):
  container[:,:,i] = raster.GetRasterBand(i+1).ReadAsArray().astype(np.float32)

In [ ]:
Final_container = (container.shape[0]*container.shape[1],container.shape[2])
Final_container_re = container[:,:,:4].reshape(Final_container)

supervised_classification = randomforest.predict(Final_container_re)
supervised_classification = supervised_classification.reshape(container[:,:,0].shape)

In [ ]:
plt.figure(figsize=[15,15])
plt.imshow(supervised_classification) # optional
plt.colorbar()

In [ ]:
xsize = np.shape(supervised_classification)[1]
ysize = np.shape(supervised_classification)[0]

print(xsize)
print(ysize)

1214
956


In [ ]:
geot = raster.GetGeoTransform()
proj = raster.GetProjectionRef()
filename = '/....tif'
xsize = np.shape(supervised_classification)[1]
ysize = np.shape(supervised_classification)[0]

driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(filename, xsize, ysize, bands = 1, eType = gdal.GDT_UInt16)
dataset.GetRasterBand(1).WriteArray(supervised_classification)
dataset.SetGeoTransform(geot)
dataset.SetProjection(proj)

dataset.FlushCache()
dataset=None

Export seagrass and sediments

In [ ]:
import os
path = '...'
files = os.listdir(path)


In [ ]:
files

['sc20190501.tif',
 'sc20200520.tif',
 'sc20190625.tif',
 'sc20210704.tif',
 'sc20210803.tif',
 'sc20211231.tif',
 'sc20181217.tif',
 'sc20190302.tif',
 'sc20191008.tif',
 'sc20200205.tif',
 'sc20200902.tif',
 'sc20210110.tif',
 'sc20210917.tif',
 'sc20190116.tif',
 'sc20190215.tif',
 'sc20210902.tif',
 'sc20191122_notgood.tif',
 'sc20210125.tif',
 'sc20220224.tif',
 'sc20201111.tif']

In [ ]:
data = gdal.Open(path+files[-1])
im_width = data.RasterXSize
im_height = data.RasterYSize
im_bands = data.RasterCount
im_geotrans = data.GetGeoTransform()
im_proj = data.GetProjection()
im_data = np.array(data.GetRasterBand(1).ReadAsArray(xoff=0,yoff=0,win_xsize=im_width,win_ysize=im_height))
seagrass = np.where(im_data <3, 1, np.nan)
sediment = np.where((im_data == 3) | (im_data==8) , 1, np.nan)
driver = gdal.GetDriverByName("GTiff")
output1 = driver.Create('/content/gdrive/My Drive/Supervised_classification/export/'+files[-1]+'_seagrass'+'.tif', im_width, im_height, 1, gdal.GDT_Float32)
output1.SetGeoTransform(im_geotrans)
output1.SetProjection(im_proj)
output1.GetRasterBand(1).WriteArray(seagrass)
output2 = driver.Create('/content/gdrive/My Drive/Supervised_classification/export/'+files[-1]+'_sediment'+'.tif', im_width, im_height, 1, gdal.GDT_Float32)
output2.SetGeoTransform(im_geotrans)
output2.SetProjection(im_proj)
output2.GetRasterBand(1).WriteArray(sediment)

0

Make it a loop to auto-run the detection model on all images

In [ ]:
for i in range(len(files)):
  data = gdal.Open(path+files[i])
  im_width = data.RasterXSize
  im_height = data.RasterYSize
  im_bands = data.RasterCount
  im_geotrans = data.GetGeoTransform()
  im_proj = data.GetProjection()
  im_data = np.array(data.GetRasterBand(1).ReadAsArray(xoff=0,yoff=0,win_xsize=im_width,win_ysize=im_height))
  seagrass = np.where(im_data <3, 1, np.nan)
  sediment = np.where((im_data == 3) | (im_data==8) , 1, np.nan)
  driver = gdal.GetDriverByName("GTiff")
  output1 = driver.Create('/content/gdrive/My Drive/Supervised_classification/export/'+files[i]+'_seagrass'+'.tif', im_width, im_height, 1, gdal.GDT_Float32)
  output1.SetGeoTransform(im_geotrans)
  output1.SetProjection(im_proj)
  output1.GetRasterBand(1).WriteArray(seagrass)
  output2 = driver.Create('/content/gdrive/My Drive/Supervised_classification/export/'+files[i]+'_sediment'+'.tif', im_width, im_height, 1, gdal.GDT_Float32)
  output2.SetGeoTransform(im_geotrans)
  output2.SetProjection(im_proj)
  output2.GetRasterBand(1).WriteArray(sediment)